In [1]:
# Run this code block just once after the kernel starts
# to change current directory to root

import os

os.chdir("..")
print(os.getcwd())

j:\GitHub\translating-images-into-maps-deep-learning


In [2]:
from configs.config_utilities import load_config

config = load_config()

In [3]:
from dataset import NuScenesDataset
from model import UNET

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from tqdm import tqdm

# IOU Evaluation for Multilabel

In [24]:
# first frame of first scene
sample_tokens = ["e3d495d4ac534d54b321f50006683844"]

train_dataset = NuScenesDataset(
    nuscenes_dir=config.nuscenes_dir,
    nuscenes_version=config.nuscenes_version,
    image_size=(200, 196),
    label_dir=config.label_dir,
    sample_tokens=sample_tokens,
)
train_loader = DataLoader(
    train_dataset,
    batch_size=1,
    num_workers=2,
    pin_memory=True,
    shuffle=True,
)

--------------------------------------------------
Loading NuScenes version v1.0-mini ...
--------------------------------------------------


In [25]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    device = "cuda"
else:
    device = "cpu"

print(f"Training on {device}...")

Training on cuda...


In [26]:
network = UNET(in_channels=3, out_channels=14)
network.to(device)

criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.Adam(network.parameters(), lr=config.lr)

In [27]:
images, labels, masks = next(iter(train_loader))
images = images.to(device)
labels = labels.to(device)

In [28]:
print(images.shape)
print(labels.shape)

torch.Size([1, 3, 196, 200])
torch.Size([1, 14, 196, 200])


In [29]:
predictions = network(images).to(device)
print(predictions.shape)

torch.Size([1, 14, 196, 200])


In [38]:
loss = criterion(predictions, labels.float()).to(device)

In [39]:
loss.item()

0.7152657508850098

In [40]:
from torchmetrics.classification import MultilabelJaccardIndex
metric = MultilabelJaccardIndex(num_labels=14).to(device)
metric(predictions, labels)

tensor(0.0187, device='cuda:0')

# IOU Evaluation for Multiclass

In [41]:
# first frame of first scene
sample_tokens = ["e3d495d4ac534d54b321f50006683844"]

train_dataset = NuScenesDataset(
    nuscenes_dir=config.nuscenes_dir,
    nuscenes_version=config.nuscenes_version,
    image_size=(200, 196),
    label_dir=config.label_dir,
    sample_tokens=sample_tokens,
    flatten_labels=True
)
train_loader = DataLoader(
    train_dataset,
    batch_size=1,
    num_workers=2,
    pin_memory=True,
    shuffle=True,
)

--------------------------------------------------
Loading NuScenes version v1.0-mini ...
--------------------------------------------------


In [12]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    device = "cuda"
else:
    device = "cpu"

print(f"Training on {device}...")

Training on cuda...


In [42]:
network = UNET(in_channels=3, out_channels=15)
network.to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(network.parameters(), lr=config.lr)

In [43]:
images, labels, masks = next(iter(train_loader))
images = images.to(device)
labels = labels.to(device)

In [44]:
print(images.shape)
print(labels.shape)

torch.Size([1, 3, 196, 200])
torch.Size([1, 196, 200])


In [45]:
predictions = network(images).to(device)
print(predictions.shape)

torch.Size([1, 15, 196, 200])


In [47]:
loss = criterion(predictions, labels.long()).to(device)

In [48]:
loss.item()

3.00421404838562

In [52]:
print(predictions.dtype)
print(labels.dtype)

torch.float32
torch.int32


In [53]:
from torchmetrics.classification import MulticlassJaccardIndex
metric = MulticlassJaccardIndex(num_classes=15).to(device)
metric(predictions, labels)

tensor(0.0031, device='cuda:0')